<a href="https://colab.research.google.com/github/Rdeandres/Game_Analytics_Classification/blob/main/RFM_Quick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Accesing the Data in Player Warehouse
This is real data, need to change for dummy data!!!!

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [3]:
#Connecting to Google Big Query
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [4]:
#Replace 'project_id' with your BigQuery project ID
from google.cloud import bigquery
client = bigquery.Client(project='ga-looker-5add7883')

# Structuring RFM values for the game

We want to apply the RFM framework to this aggregate dataset and answer the following questions:

Is there a correlation between Recency, Frequency, and Monetary Value?
How should we define our most valuable customers (MVC)? What percentage of our customers are the most valuable customers?
Are we able to create distinguishing segments and design campaigns accordingly to improve customer engagement and/or monetization?

In [13]:
#need to select by game id 

#country_code can be added 

sql_query = ('''

                SELECT 
                player_id,
                min(DATETIME (install_ts)) as subscription_date,
                max(DATETIME_DIFF(DATETIME ('2022-02-20'), DATETIME (install_ts), DAY)) as maturity,
                sum(ad_ilrd_revenue) as value,
                min(cohort_week) as subscription_bom,
                count(distinct checkpoint) as frequency,
                max(DATETIME_DIFF(DATETIME ('2022-02-20'), DATETIME (checkpoint), DAY)) as recency

                
                FROM  `ga-looker-5add7883.checkpoints.player_checkpoint`
                where checkpoint>= '2022-01-03' and checkpoint <= '2022-02-20'  and game_id=135152
                group by 1
         
                ''')
ret = client.query(sql_query).to_dataframe()
ret.head()

,player_id,subscription_date,maturity,value,subscription_bom,frequency,recency
0,6dae651cc787216541b2824dfcc7221d,2021-05-26,270,NaN,2021-05-24,9,48
1,6f4ac5dca13dba333f305c8726df768b,2021-06-17,248,NaN,2021-06-14,4,29
2,a9e8dd927e7be2247453b1efb29dcdc3,2021-06-22,243,0.044483,2021-06-21,1,27
3,30e5bf473599303dfdd6f8a47db7eba2,2021-08-03,201,0.945508,2021-08-02,3,47
4,f8957327d0736c132c6646d5ec32c8fe,2021-08-26,178,NaN,2021-08-23,7,37


In [14]:
ret["frequencyRank"] = ret["frequency"].rank(method='first')

#Use qcut on the rank to determine equal sized quantiles. Below example creates deciles (bins=10)
ret["FrequencyScore"] = pd.qcut(ret["frequencyRank"].values, 10).codes

In [15]:
#descriptive
agg_freq=ret[["frequency"]].groupby(ret["FrequencyScore"]).min()
print(agg_freq)

                frequency
FrequencyScore           
0                       1
1                       1
2                       1
3                       1
4                       1
5                       1
6                       2
7                       2
8                       3
9                       4


In [22]:
#with recency the more recent the better, so we rank in the opposite direcction
ret["recencyRank"] = ret["recency"].rank(method='first')

ret["RecencyScore"] = pd.qcut(ret["recencyRank"].values, 10).codes

def assign_recscore(row):


    if row >= 9:

        result = 0

    elif row >= 8:

        result = 1

    elif row >= 7:

        result = 2

    elif row >= 6:

        result = 3

    elif row >= 5:

        result = 4

    elif row >= 4:

        result = 5

    elif row >= 3:

        result = 6

    elif row >= 2:

        result = 7

    elif row >= 1:

        result = 8

    elif row >= 0:

        result = 9

    

    else:


        result = -1


    return result




ret["RecencyScorev2"] = ret['RecencyScore'].apply(assign_recscore)




ret["valueRank"] = ret["value"].rank(method='first')
#Use qcut on the rank to determine equal sized quantiles. Below example creates deciles (bins=10)
ret["ValueScore"] = pd.qcut(ret["valueRank"].values, 10).codes

In [23]:
ret["RFM"] = ret["RecencyScorev2"] + ret["FrequencyScore"] + ret["ValueScore"]

In [25]:
#descriptive, the more recent the higher the score
agg_rec=ret[["recency"]].groupby(ret["RecencyScorev2"]).min()
print(agg_rec)

                recency
RecencyScorev2         
0                    46
1                    43
2                    40
3                    35
4                    30
5                    26
6                    20
7                    11
8                     6
9                     0


In [26]:
ret.head()

,player_id,subscription_date,maturity,value,subscription_bom,frequency,recency,frequencyRank,FrequencyScore,recencyRank,RecencyScore,valueRank,ValueScore,RFM,RecencyScorev2
0,6dae651cc787216541b2824dfcc7221d,2021-05-26,270,NaN,2021-05-24,9,48,4166184.0,9,3993761.0,9,NaN,-1,8,0
1,6f4ac5dca13dba333f305c8726df768b,2021-06-17,248,NaN,2021-06-14,4,29,3649781.0,8,1898477.0,4,NaN,-1,12,5
2,a9e8dd927e7be2247453b1efb29dcdc3,2021-06-22,243,0.044483,2021-06-21,1,27,1.0,0,1694183.0,4,202325.0,0,5,5
3,30e5bf473599303dfdd6f8a47db7eba2,2021-08-03,201,0.945508,2021-08-02,3,47,3200304.0,7,3822347.0,9,1129282.0,4,11,0
4,f8957327d0736c132c6646d5ec32c8fe,2021-08-26,178,NaN,2021-08-23,7,37,4092439.0,9,2670545.0,6,NaN,-1,11,3


In [28]:
#on paying events, min and max per player

summary=ret.groupby("RFM").agg(
        min_freq=pd.NamedAgg(column='frequency', aggfunc='min'),
        max_freq=pd.NamedAgg(column='frequency', aggfunc='max'),
        average_freq=pd.NamedAgg(column='frequency', aggfunc=np.mean),
        median_freq=pd.NamedAgg(column='frequency', aggfunc=np.median),

        min_rec=pd.NamedAgg(column='recency', aggfunc='min'),
        max_rec=pd.NamedAgg(column='recency', aggfunc='max'),
        average_rec=pd.NamedAgg(column='recency', aggfunc=np.mean),
        median_rec=pd.NamedAgg(column='recency', aggfunc=np.median),

        min_val=pd.NamedAgg(column='value', aggfunc='min'),
        max_val=pd.NamedAgg(column='value', aggfunc='max'),
        average_val=pd.NamedAgg(column='value', aggfunc=np.mean),
        median_val=pd.NamedAgg(column='value', aggfunc=np.median),

        min_mat=pd.NamedAgg(column='maturity', aggfunc='min'),
        max_mat=pd.NamedAgg(column='maturity', aggfunc='max'),
        average_mat=pd.NamedAgg(column='maturity', aggfunc=np.mean),
        median_mat=pd.NamedAgg(column='maturity', aggfunc=np.median),

    )

display(summary)

,min_freq,max_freq,average_freq,median_freq,min_rec,max_rec,average_rec,median_rec,min_val,max_val,average_val,median_val,min_mat,max_mat,average_mat,median_mat
RFM,,,,,,,,,,,,,,,,
-1,1,1,1.000000,1.0,47,48,47.557389,48.0,NaN,NaN,NaN,NaN,53,328,142.257677,115.0
0,1,1,1.000000,1.0,44,48,45.170913,45.0,0.000098,0.066206,0.024413,0.019354,53,328,149.595933,129.0
1,1,1,1.000000,1.0,41,48,42.356803,42.0,0.000033,0.246300,0.070629,0.049600,53,328,158.289838,146.0
2,1,1,1.000000,1.0,36,48,38.450095,38.0,0.000195,0.517240,0.149027,0.098373,53,328,160.538482,149.0
3,1,1,1.000000,1.0,31,48,38.922422,35.0,0.000026,0.856958,0.206511,0.092102,46,328,120.274613,75.0
4,1,2,1.079380,1.0,27,48,45.034330,46.0,0.000031,1.307504,0.178992,0.049463,43,328,62.237490,47.0
5,1,2,1.110318,1.0,21,48,40.212467,42.0,0.000020,1.941395,0.169057,0.053904,35,328,68.981158,42.0
6,1,3,1.445710,1.0,12,48,38.529506,39.0,0.000013,2.926203,0.176677,0.056251,30,328,75.456711,47.0
7,1,4,1.835160,2.0,7,48,37.429341,43.0,0.000010,4.720301,0.131099,0.039531,20,328,74.414465,46.0


# Inputting bounds for segmentation 

In [38]:
ret["one_week_old"] = ret["maturity"]<=7 # installed in the last week
ret["more_than_two_orders"] = ret["frequency"]>2 # Logged in more than twice
ret["value_higher_than_2"] = ret["value"]>2# Sum of value higher than 1
ret["recency_week"] = ret["recency"]>7# Sum of value higher than 1
ret.groupby(["one_week_old","recency_week", "more_than_two_orders", "value_higher_than_2"]).count()["recency"]

one_week_old  recency_week  more_than_two_orders  value_higher_than_2
False         False         False                 False                    37791
                                                  True                      8451
                            True                  False                      632
                                                  True                       472
              True          False                 False                  2280036
                                                  True                    453370
                            True                  False                   556349
                                                  True                    407260
True          False         False                 False                   281362
                                                  True                    139293
                            True                  False                    16161
                                       

In [56]:
ret["Segment"]= '4.Low_value'
ret.loc[ret['maturity'] <= 7, 'Segment'] = '5.First_week'
ret.loc[(ret['maturity'] > 7) & (ret['value'] > 2) , 'Segment'] = '2.Median_earn'
ret.loc[(ret['maturity'] > 7) & (ret['frequency'] > 2), 'Segment'] = '3.Median_frequent'
ret.loc[(ret['maturity'] > 7) & (ret['value'] > 2 ) & (ret['frequency'] > 2), 'Segment'] = '1.High_value'

# Segmentation profile

In [59]:
#on paying events, min and max per player

Segment_summary=ret.groupby("Segment").agg(
        min_freq=pd.NamedAgg(column='frequency', aggfunc='min'),
        max_freq=pd.NamedAgg(column='frequency', aggfunc='max'),
        average_freq=pd.NamedAgg(column='frequency', aggfunc=np.mean),
        median_freq=pd.NamedAgg(column='frequency', aggfunc=np.median),

        min_rec=pd.NamedAgg(column='recency', aggfunc='min'),
        max_rec=pd.NamedAgg(column='recency', aggfunc='max'),
        average_rec=pd.NamedAgg(column='recency', aggfunc=np.mean),
        median_rec=pd.NamedAgg(column='recency', aggfunc=np.median),

        min_val=pd.NamedAgg(column='value', aggfunc='min'),
        max_val=pd.NamedAgg(column='value', aggfunc='max'),
        average_val=pd.NamedAgg(column='value', aggfunc=np.mean),
        median_val=pd.NamedAgg(column='value', aggfunc=np.median),

        min_mat=pd.NamedAgg(column='maturity', aggfunc='min'),
        max_mat=pd.NamedAgg(column='maturity', aggfunc='max'),
        average_mat=pd.NamedAgg(column='maturity', aggfunc=np.mean),
        median_mat=pd.NamedAgg(column='maturity', aggfunc=np.median),
        count_players=pd.NamedAgg(column='maturity', aggfunc=np.count_nonzero)
    )

display(Segment_summary)


,min_freq,max_freq,average_freq,median_freq,min_rec,max_rec,average_rec,median_rec,min_val,max_val,average_val,median_val,min_mat,max_mat,average_mat,median_mat,count_players
Segment,,,,,,,,,,,,,,,,,
1.High_value,3,48,5.005967,4.0,2,48,28.129428,29.0,2.000000,2181.937037,12.933988,5.801661,8,328,41.887213,30.0,407732
2.Median_earn,1,2,1.450471,1.0,0,48,23.573434,25.0,2.000005,416.464578,7.957256,4.299178,8,328,42.007596,28.0,461821
3.Median_frequent,3,48,4.238295,4.0,2,48,37.437225,42.0,0.000010,2.000000,0.837674,0.776092,8,328,66.297095,44.0,556981
4.Low_value,1,2,1.271994,1.0,0,48,32.197872,35.0,0.000006,2.000000,0.615855,0.460829,8,328,63.491147,41.0,2317827
5.First_week,1,8,1.528898,1.0,0,7,3.704482,4.0,0.000010,859.399961,3.927428,1.387620,0,7,3.704482,4.0,425282


# Forecast based on segmentation

In [ ]:

sql_query = ('''

                SELECT 
                player_id,
                min(DATETIME (install_ts)) as subscription_date,
                max(DATETIME_DIFF(DATETIME ('2022-02-20'), DATETIME (install_ts), DAY)) as maturity,
                sum(ad_ilrd_revenue) as value,
                min(cohort_week) as subscription_bom,
                count(distinct checkpoint) as frequency,
                max(DATETIME_DIFF(DATETIME ('2022-02-20'), DATETIME (checkpoint), DAY)) as recency

                
                FROM  `ga-looker-5add7883.checkpoints.player_checkpoint`
                where checkpoint>= '2022-01-03' and checkpoint <= '2022-02-20'  and game_id=135152
                group by 1
         
                ''')
ret = client.query(sql_query).to_dataframe()
ret.head()